In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input": "HI!"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [3]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4
)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})
  
add_message("HI!", "How are you?")
add_message("I'm good, how are you?", "I'm good too!")
add_message("That's great!", "Yes it is!")
add_message("I'm glad you're doing well", "Thank you!")

In [4]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='HI!'),
  AIMessage(content='How are you?'),
  HumanMessage(content="I'm good, how are you?"),
  AIMessage(content="I'm good too!"),
  HumanMessage(content="That's great!"),
  AIMessage(content='Yes it is!'),
  HumanMessage(content="I'm glad you're doing well"),
  AIMessage(content='Thank you!')]}

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()